In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torchaudio pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
%cd /content/drive/MyDrive/learnpal/speek

/content/drive/MyDrive/learnpal/speek


In [ ]:
# !apt-get install ffmpeg
# import os
# import subprocess
# fp='/content/drive/MyDrive/learnpal'
# audio_dir = fp+'/speek/audio2/'
# audio_dir2 = fp+'/speek/audio/'
# m4a_files = [f for f in os.listdir(audio_dir2) if f.endswith('.m4a')]

# for m4a_file in m4a_files:
#     wav_file = m4a_file.replace('.m4a', '.wav')
#     input_path = os.path.join(audio_dir2, m4a_file)
#     output_path = os.path.join(audio_dir, wav_file)
#     command = ['ffmpeg', '-i', input_path, output_path]
#     subprocess.run(command)

In [4]:
import pandas as pd

fp='/content/drive/MyDrive/learnpal'
# Path to the transcripts
transcripts_path = fp+'/speek/Auditory.xlsx'

# Read the Excel file
transcripts_df = pd.read_excel(transcripts_path)
transcripts_df = transcripts_df.dropna(subset=['transcript'])
transcripts_df = transcripts_df[transcripts_df['transcript'].str.strip() != '']
transcripts_df = transcripts_df.reset_index(drop=True)

# Display the first few rows to verify
print(transcripts_df.head())


   id   filename                   transcript
0   1  Gs1q1.m4a                   හැත්ත හයයි
1   2  Gs1q2.m4a  දොඩම් ගෙඩි එකසිය තිස් හතරයි
2   3  Gs1q3.m4a       නාරං ගෙඩි පන්සිය දහයයි
3   4  Gs1q4.m4a   පැන්සල් හා රසිය විසි නවයයි
4   5  Gs1q5.m4a       බනිස් අටසිය හැත්ත අටයි


In [5]:
import os
from glob import glob

# Path to audio files
audio_dir = fp+'/speek/audio2/'

# List all audio files
audio_files = glob(os.path.join(audio_dir, '*.wav'))
audio_filenames = [os.path.basename(f) for f in audio_files]

# Create data entries
data = []
for idx, row in transcripts_df.iterrows():
    audio_filename = row['filename']  # Adjust if your column name is different
    transcript = row['transcript']
    audio_filename=audio_filename.replace('.m4a','.wav')
    if audio_filename in audio_filenames:
        audio_path = os.path.join(audio_dir, audio_filename)
        data.append({'audio': audio_path, 'transcript': transcript})
    else:
        print(f"Warning: Audio file {audio_filename} not found.")


In [6]:
from datasets import Dataset, Audio

# Create the Dataset
dataset = Dataset.from_list(data)

# Cast the 'audio' column to Audio type
dataset = dataset.cast_column('audio', Audio(sampling_rate=16000))


In [8]:
from transformers import WhisperProcessor

# Load the Whisper processor
processor = WhisperProcessor.from_pretrained("lonewol7f/whisper-small-sinhala-12k", language="Sinhala", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [9]:
def preprocess_function(batch):
    # Load and resample audio data
    audio = batch["audio"]

    # Compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text to label ids
    labels = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=448
    ).input_ids

    # Check if labels are empty
    if len(labels) == 0:
        print(f"Warning: Empty labels for transcript: {batch['transcript']}")
        labels = [processor.tokenizer.eos_token_id]

    batch["labels"] = labels

    return batch

processed_dataset = dataset.map(
    preprocess_function,
    remove_columns=dataset.column_names,
    num_proc=1  # Adjust based on your CPU cores
)

Map:   0%|          | 0/1349 [00:00<?, ? examples/s]

In [10]:
# Optional: Verify processed dataset
def check_empty_examples(dataset):
    for idx in range(len(dataset)):
        example = dataset[idx]
        if len(example['input_features']) == 0:
            print(f"Empty input_features at index {idx}")
        if len(example['labels']) == 0:
            print(f"Empty labels at index {idx}")

check_empty_examples(processed_dataset)

In [11]:
# Split dataset (e.g., 90% train, 10% test)
split_dataset = processed_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [19]:
from transformers import (
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import torch

model = WhisperForConditionalGeneration.from_pretrained("lonewol7f/whisper-small-sinhala-12k")

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
    language='Sinhala',
    task='transcribe'
)
model.config.suppress_tokens = None

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-sinhala-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=3,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=1e-5,
    predict_with_generate=True,
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# Define the data collator (if not available, define it manually)
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    padding: Union[bool, str] = True
    max_length: Union[int, None] = None
    max_length_labels: Union[int, None] = None
    pad_to_multiple_of: Union[int, None] = None
    pad_to_multiple_of_labels: Union[int, None] = None

    def __call__(self, features):
        # Split inputs and labels
        input_features = [feature["input_features"] for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad inputs
        batch = self.processor.feature_extractor.pad(
            {"input_features": input_features},
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Pad labels
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=self.padding,
            max_length=self.max_length_labels,
            pad_to_multiple_of=self.pad_to_multiple_of_labels,
            return_tensors="pt",
        )

        # Replace padding with -100 to ignore in loss
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["attention_mask"].ne(1), -100
        )

        batch["labels"] = labels

        return batch

In [22]:
!pip install evaluate
!pip install jiwer

In [23]:
import evaluate

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

wer_metric = evaluate.load("wer")  # Updated line

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 in labels as we can't decode them
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode predictions and references
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Filter out empty predictions or labels
    filtered_pred_str = []
    filtered_label_str = []
    for p, l in zip(pred_str, label_str):
        if p.strip() == '' or l.strip() == '':
            continue
        filtered_pred_str.append(p)
        filtered_label_str.append(l)

    if len(filtered_pred_str) == 0:
        print("Warning: No valid predictions to compute metrics.")
        return {"wer": None}

    # Compute WER
    wer = wer_metric.compute(predictions=filtered_pred_str, references=filtered_label_str)
    return {"wer": wer}

In [24]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-24-b7ceb9b01a3c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [25]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
100,0.060400,0.044702,0.141844
200,0.021900,0.028184,0.099291


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=228, training_loss=0.08608527102491312, metrics={'train_runtime': 1095.7521, 'train_samples_per_second': 3.324, 'train_steps_per_second': 0.208, 'total_flos': 1.05102802796544e+18, 'train_loss': 0.08608527102491312, 'epoch': 3.0})

In [34]:
model.save_pretrained("./whisper-small-sinhala-finetuned")
processor.save_pretrained("./whisper-small-sinhala-finetuned")

[]

In [36]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio

# Load the fine-tuned model and processor
model = WhisperForConditionalGeneration.from_pretrained("./whisper-small-sinhala-finetuned")
processor = WhisperProcessor.from_pretrained("./whisper-small-sinhala-finetuned")

# Set the language and task
language = "Sinhala"
task = "transcribe"

# Update the model's generation configuration
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
model.config.suppress_tokens = None


In [40]:
!apt-get install -y ffmpeg
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Import libraries
from IPython.display import Javascript, display, Audio
from google.colab import output
from base64 import b64decode
from pydub import AudioSegment


# Function to record audio
def record_audio(filename="audio.webm", duration=5):
    js = Javascript(
        """
    async function recordAudio() {
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time));
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader();
        reader.onloadend = () => resolve(reader.result);
        reader.readAsDataURL(blob);
      });

      var stream = await navigator.mediaDevices.getUserMedia({audio: true});
      var recorder = new MediaRecorder(stream);
      var data = [];

      recorder.ondataavailable = event => data.push(event.data);
      recorder.start();

      console.log("Now say the word:")

      await sleep(%d);

      recorder.stop();

      await sleep(1000);

      stream.getAudioTracks()[0].stop();

      var blob = new Blob(data, {type: 'audio/webm'});
      var text = await b2text(blob);
      return text;
    }
    """
        % (duration * 1000)
    )

    display(js)
    data = output.eval_js("recordAudio()")
    binary = b64decode(data.split(",")[1])
    with open(filename, "wb") as f:
        f.write(binary)
    print("Audio recorded and saved as {}".format(filename))


# Record the audio
record_audio()


# Convert to WAV format (optional)
def convert_audio(input_file="audio.webm", output_file="audio.wav"):
    audio = AudioSegment.from_file(input_file, format="webm")
    audio.export(output_file, format="wav")
    print("Audio converted and saved as {}".format(output_file))


convert_audio()

# Play the audio
display(Audio("audio.wav"))

<IPython.core.display.Javascript object>

Audio recorded and saved as audio.webm
Audio converted and saved as audio.wav


In [46]:
# Load the audio file
audio_file = "audio.wav"
speech_array, sampling_rate = torchaudio.load(audio_file)

# Resample the audio to 16 kHz if necessary
if sampling_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
    speech_array = resampler(speech_array)

# Convert to mono channel if necessary
if speech_array.shape[0] > 1:
    speech_array = speech_array.mean(dim=0)

# Prepare the input features
input_features = processor.feature_extractor(
    speech_array.numpy(), sampling_rate=16000, return_tensors="pt"
).input_features

# Move model and inputs to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
input_features = input_features.to(device)

# Generate transcription
with torch.no_grad():
    generated_ids = model.generate(input_features)

# Decode the transcription
transcription = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Transcription:", transcription)

Transcription: ඇපල් ගෙඩි හයසිය පනස් තුනයි
